# Documentation - piRNA pipe



## Report

### 1. Expression 

1.1 Distribution of small RNAs, (Julius et al., 2008, Science, Figure2A)    
1.2 plots, read coverage on TE  




## 1 Purpose  

For small RNA seqeuencing analysis (total, piwi-IP seq, etc). 




## 2 workflow  


### 2.1 pre-analysis  

+ Main steps 

  - remove structural RNAs (rRNA, tRNA, snRNA, snoRNA, ..., check rRNA%) 
  - remove miRNAs (miRNA hairpin, 0 mismatch, unique + multi)  
  - split by length ([23,29], others)  
  - map to TE consensus (0 mismatch, unique) 
  - map to piRNA clusters (0 mismatch, unique)  
  - map to genome (0 mismatch, unique + multi)  
  
+ Extra  

  - read size distribution  
  - evaluate 1U-10A   
  - check overlap between samples  
  
+ To-do  
  
  - extract siRNAs (21nt)  
  - ping-pong signature score 
  - correlation between samples
  - compare with piRBase (v2.0), other published data, overlap (+)
  
  - TE read count
 



## 3 Analysis  


### 3.1 main  

  - `piRNA_pipe.py`  
  
  
  
### 3.2 modules  

  - `fragsize.py` - calculate the distribution of read length  
  
  - `u1a10.py` - calculate the U1A10 content  
  
  - `fx_overlap.py` - check the overlap between two fastx file





### 3.3 Directory structure 

```
├── 00.raw_data
├── 01.clean_data
├── 02.collapse
├── 03.overlap
├── 04.smRNA
├── 05.miRNA
├── 06.size_exclude
├── 06.size_select
├── 07.te
├── 08.piRNA_cluster
├── 09.genome
├── 10.unmap
├── 11.stat
└── 12.report

```


### 3.4 Output files



 
## x. Change log




```
piRNA analysis (small RNAseq) 


date: 2020-12-27

1. support collapsed reads, counting, RNA seqs, reads


date: 2020-12-23
Author: Ming Wang

# flowchart of piRNA analysis
1. remove structural RNAs (uniqe + multi)
2. remove miRNAs (unique + multi) 
3. remove reads not in [23-29] nt 
4. collapse reads: consider 1-23nt only, allow 1-2 at 3' differ (2019-11-26)
5. split into 4 groups: (1U+/-,10A+/-;)
6. map to TE consensus, (unique + multi), only 1U_not_10A
7. map to genome (unique + multi) 
Functions:
rename fastq reads: piR0000001-0000001
piR (piRNA), (piRNA number) {reads number}
mapping, unique + multiple/unique


version: 2020-07-28
update:
1. remove temp fastq files
2. gzip fastq files

version: 2020-07-25
update:
1. collapse reads 


date: 2020-07-23
in brief:

1. remove 3' adapters  
2. remove structural RNAs (tRNAs, rRNAs) (unique + multiple)     
3. filt by length, 23-29 nt   
4. collapse reads, only compare 1-23 nt (*)  
   collapse reads, (regular)  
   trim reads to 23nt from 3' end  

5. map to TE consensus (unique, multiple, unique + multiple)   
6. map to piRNA clusters (unique, multiple, unique + multiple)   
7. map to genome (not-TE, no-piRNAcluster)    
8. Overall, map to genome
```





In [ ]:
%reload_ext autoreload
%autoreload 2
from piRNA_pipe import *

args = {
    'fq': 'test/a.fq.gz',
    'outdir': 'test/results',
    'collapse': True,
    'subject': 'test/b.fq.gz',
    'force_overlap': True,
    'ov_type': 1,
}
# p = pipe(**args).run()

args = {
    'query': 'test/a.fq',
    'subject': 'test/b.fq',
    'outdir': 'test/overlap',
    'mm': 2
}
OverlapFq(**args).run()
# overlap_fq('test/a.fq', 'test/b.fq', 'test/overlap2')

In [ ]:
# U1A10

%reload_ext autoreload
%autoreload 2
from fastx import *

fx = ['test/a.fq.gz', 'test/b.fq.gz']

FxU1A10(fx).run()


In [ ]:
# Alignemt
%reload_ext autoreload
%autoreload 2
import pyfastx
from align import *

args = {
    'fx': 'test/a.fq.gz',
    'index': '/data/yulab/wangming/data/genome/dm6/bowtie_index/miRNA',
    'outdir': 'test/bbbbbb',
    'unique': 'multi'
}

Align(**args).run()


In [ ]:
# PiRNApipe

%reload_ext autoreload
%autoreload 2
import pyfastx
from piRNA_pipe import *

args = {
    'fq': 'test/dme.1m.fa.gz',
    'outdir': 'test/aaaaaa',
    'genome': 'dm6',
    'subject_list': 'test/a.fq.gz'
}

p1 = PiRNApipe(**args)
# p1.run()
# p2 = overlap_subject(p1)

for k, v in p1.__dict__.items():
    print('{}: {}'.format(k, v))


In [ ]:
# stat
%reload_ext autoreload
%autoreload 2
from qc import *
import qc
from hiseq.fragsize.fragsize import BamFragSize

# x = 'test/aaaaaa/a'
# y = x + '/00.raw_data'
# df = PiRNApipeStat(x).run()
# df

# x = '/data/yulab/wangming/work/yu_2020/public_data/2016_molcell_siomi/results/smRNA/smRNAseq_H1_KD_Piwi_IP_rep1'

# p = PiRNApipeStat(x)
# # p.run()
# # p.report()
# bam_list = p.list_bam(u1a10=True)
# # bam_list
# for b in bam_list:
#     try:
#         FxFragSize(b).run()
#     except:
#         print("!AAAA-1", b)

x = '/data/yulab/wangming/work/yu_2020/public_data/2008_science_greg/results/piRNA_pipe/Embryo_0_2hrs_rep1_w1118_0_2hr_Emb'
p = PiRNApipeStat(x)
# # p.run()
# dirs = p.list_dir(u1a10=False, group=False)
# for d in dirs:
#     try: 
#         p.stat_dir(d)
#     except:
#         print('!AAAA-1', d)
d = '/data/yulab/wangming/work/yu_2020/public_data/2008_science_greg/results/piRNA_pipe/Embryo_0_2hrs_rep1_w1118_0_2hr_Emb/00.raw_data'
p.stat_dir(d)

In [ ]:
def stat_dir(d):
    """Create summary for level-1 directory
    number of reads, for each category
    total: 
    U1_A10: 
    """
    h = ['sample', 'num_reads', 'num_seqs']
    dfx = pd.DataFrame(columns=h)
    # level-1: dirs
    group = os.path.basename(d)
    t1 = self.list_count_toml(d)
    if len(t1) == 1:
        df1 = self.load_toml(t1[0])
    else:
        df1 = dfx
    df1.columns = h
    df1['u1a10'] = 'all'
    # level-2: U1A10
    t2 = self.list_count_toml(d+'/U1_A10')
    frames = [self.load_toml(i) for i in t2]
    if len(frames) > 0:
        df2 = pd.concat(frames, axis=0)
        df2.columns = h
        df2[['sample', 'u1a10']] = df2['sample'].str.split('.', 1, expand=True)
    else:
        df2 = dfx
        df2['u1a10'] = 'all'
    # combine
    df = pd.concat([df1, df2], 'index')
    df['group'] = group
    # stat
    if len(t1) > 0:
        d_csv = os.path.join(d, 'fx_stat.csv')
        df.to_csv(d_csv, index=False)
    return df

d = '/data/yulab/wangming/work/yu_2020/public_data/2008_science_greg/results/piRNA_pipe/Embryo_0_2hrs_rep1_w1118_0_2hr_Emb/07.te/uniuq/U1_A10'
t2 = p.list_count_toml(d+'/U1_A10')
frames = [p.load_toml(i) for i in t2]
# if len(frames) > 0:
#     df2 = pd.concat(frames, axis=0)
#     df2.columns = h
#     df2[['sample', 'u1a10']] = df2['sample'].str.split('.', 1, expand=True)
# else:
#     df2 = dfx
#     df2['u1a10'] = 'all'
# t1 = p.list_count_toml(d)
# df1 = p.load_toml(t1[0])
# df1
pd.concat(frames, axis=0)

In [ ]:
a = ['abc.txt.gz', 'a.fq.gz', 'b.unmap.fq.gz']


def get_fx_name(x):
    """Parse the name of fastx
    a.fq
    a.fq.gz
    a.unmap.fq
    a.unmap.fq.gz
    """
    if isinstance(x, str):
        x_name = os.path.basename(x)
        # remove .gz
        if x_name[-3:] == '.gz':
            x_name = x_name[:-3]
        # remove ext:
        x_name = os.path.splitext(x_name)[0]
        # remove unmap
        if x_name[-6:] == '.unmap':
            x_name = x_name[:-6]
        return x_name
    
list(map(get_fx_name, a))

In [11]:
%reload_ext autoreload
%autoreload 2
from piRNA_pipe import *
from utils import *
from track_file import *

x = ['test/aaaaaa/a', 'test/aaaaaa/b']


# run_bam_to_bw(x, 'te')
# get_x_file(x, 'bigwig', check_exists=False)

bw = [get_x_file(i, 'bigwig', 'te') for i in x]
outdir = 'test/ccc'

get_tracks(bw, outdir=outdir)


--------------------------------------------------------------------------------
track_file: /data/yulab/wangming/work/devel_pipeline/pirna_seq/scripts/test/aaaaaa/a/07.te/unique/a.bigwig,
/data/yulab/wangming/work/devel_pipeline/pirna_seq/scripts/test/aaaaaa/b/07.te/unique/b.bigwig
outdir: test/ccc
region_list: /data/yulab/wangming/work/devel_pipeline/pirna_seq/scripts/test/aaaaaa/a/07.te/unique/a.bigwig
--------------------------------------------------------------------------------
1/126 - Idefix
[2021-03-12 11:47:49 INFO] get_track() skipped, file exists: /data/yulab/wangming/work/devel_pipeline/pirna_seq/scripts/test/ccc/Idefix/Idefix_1_7411.png
